In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

In [2]:
### Configuración de Chrome ###
c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
c_options.add_argument("--disable-blink-features=AutomationControlled")
c_driver = webdriver.Chrome(options=c_options)

In [3]:
# Abrir AutoScout24 en la sección de Tesla
url_autoscout = "https://www.autoscout24.es/lst/tesla"
c_driver.get(url_autoscout)
time.sleep(3)

In [4]:
# Aceptar cookies si es necesario
try:
    boton_cookies = c_driver.find_element(By.CLASS_NAME, "_consent-accept_1lphq_114")
    boton_cookies.click()
    time.sleep(2)
except NoSuchElementException:
    print("No se encontró el botón de cookies. Continuando...")

In [5]:
### Función para obtener los artículos ###
def obtener_articulos_pagina():
    try:
        # Buscar todos los artículos de coches en la página
        articulos = c_driver.find_elements(By.TAG_NAME, "article")
        return articulos
    except NoSuchElementException:
        print("No se encontraron artículos en esta página.")
        return []

In [6]:
def generar_diccionario_articulo(articulo):
    """Genera un diccionario con los datos de un artículo, incluyendo color exterior."""
    diccionario = {}
    try:
        # Modelo (atributo 'data-model')
        diccionario['modelo'] = articulo.get_attribute("data-model").capitalize()

        # Kilometraje
        try:
            kilometraje = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-mileage_road"]').text
            diccionario['kilometraje'] = kilometraje
        except NoSuchElementException:
            diccionario['kilometraje'] = "No especificado"
        ##DataGrid_defaultDdStyle__3IYpG DataGrid_fontBold__RqU01
        # Precio
        #DataGrid_defaultDdStyle__3IYpG 
        anio = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-calendar"]').text
        print(anio)
        diccionario['year'] = anio
        try:
            precio = articulo.find_element(By.CSS_SELECTOR, '[data-testid="regular-price"]').text
            diccionario['precio'] = precio
        except NoSuchElementException:
            diccionario['precio'] = "No especificado"

        # Navegar a la página del artículo para extraer el color exterior
        try:
            ActionChains(c_driver).key_down(Keys.CONTROL).click(articulo).key_up(Keys.CONTROL).perform()
            time.sleep(3)  # Esperar a que cargue la nueva pestaña

            # Cambiar a la nueva pestaña
            c_driver.switch_to.window(c_driver.window_handles[-1])

            # Buscar el color exterior en la sección "Color y Tapicería"
            try:
                color_seccion = c_driver.find_element(By.XPATH, "//section[@data-cy='color-section']")
                color = color_seccion.find_element(By.XPATH, ".//dd").text
                diccionario['color'] = color
            except NoSuchElementException:
                diccionario['color'] = "No especificado"

            # Cerrar la pestaña actual y volver a la pestaña principal
            c_driver.close()
            c_driver.switch_to.window(c_driver.window_handles[0])
        except Exception as e:
            print(f"Error al extraer el color: {e}")
            diccionario['color'] = "No especificado"

    except Exception as e:
        print(f"Error procesando artículo: {e}")
        return None

    return diccionario

In [7]:
def extraer_datos_tesla():
    """Extrae datos de todas las páginas navegando en AutoScout24."""
    lista_datos = []
    while True:
        # Obtener artículos de la página actual
        articulos = obtener_articulos_pagina()
        for articulo in articulos:
            datos_articulo = generar_diccionario_articulo(articulo)
            if datos_articulo:
                lista_datos.append(datos_articulo)

        # Intentar pasar a la siguiente página
        try:
            # Usar aria-label para identificar el botón "Siguiente"
            boton_siguiente = WebDriverWait(c_driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Ir a la página siguiente']"))
            )
            c_driver.execute_script("arguments[0].scrollIntoView();", boton_siguiente)  # Asegurar que el botón es visible
            boton_siguiente.click()
            print("Pasando a la siguiente página...")
            time.sleep(3)
        except TimeoutException:
            print("No hay más páginas disponibles.")
            break
        except NoSuchElementException:
            print("No se encontró el botón de siguiente página.")
            break

    return lista_datos

In [8]:
### Ejecutar el scraping ###
print("Iniciando el scraping de Tesla en AutoScout24...")
datos_tesla = extraer_datos_tesla()

Iniciando el scraping de Tesla en AutoScout24...
07/2019
05/2015
09/2024
03/2020
06/2023
05/2018
03/2017
06/2023
09/2020
09/2023
02/2020
12/2022
08/2021
12/2016
09/2019
11/2020
01/2020
01/2013
12/2022
03/2022
Pasando a la siguiente página...
11/2023
02/2017
06/2017
08/2015
09/2019
05/2019
02/2023
03/2018
12/2019
11/2023
01/2019
01/2020
11/2015
01/2024
04/2017
09/2020
11/2023
07/2019
01/2022
03/2018
Pasando a la siguiente página...
10/2019
08/2019
06/2019
12/2023
02/2021
10/2016
08/2023
01/2019
09/2019
02/2021
12/2018
09/2016
03/2022
11/2017
07/2017
01/2023
06/2022
01/2023
07/2019
07/2019
Pasando a la siguiente página...
05/2021
03/2024
01/2020
01/2020
01/2020
01/2023
01/2018
01/2019
05/2017
01/2021
12/2018
09/2020
05/2020
03/2022
05/2023
06/2018
12/2021
03/2015
01/2019
09/2023
Pasando a la siguiente página...
03/2015
01/2019
09/2023
11/2015
01/2021
07/2014
03/2024
03/2024
09/2020
09/2021
04/2015
03/2021
03/2023
09/2019
09/2020
10/2020
03/2017
04/2019
01/2022
01/2019
Pasando a la siguie

In [9]:
# Guardar los datos en un archivo CSV
df = pd.DataFrame(datos_tesla)
df.to_csv("tesla_autoscout24.csv", index=False, encoding="utf-8-sig")
print("Datos guardados en 'tesla_autoscout24.csv'.")

Datos guardados en 'tesla_autoscout24.csv'.


In [10]:
# Cerrar el navegador
c_driver.quit()